In [37]:
import torch
from torch_geometric.utils import to_dense_adj

from data import get_dataset

In [53]:
graphs, idx_train, idx_val, idx_test, idx_dropped = get_dataset(
    dataset="LINUX",
    pe_dim=2,
    path="../../data/nagphormer/",
    train_split=1.0,
)

/DATATWO/users/burouj/miniconda3/envs/nag-sim/lib/python3.8/site-packages/dgl/heterograph.py:3719: DGLWarning: DGLGraph.adjacency_matrix_scipy is deprecated. Please replace it with:

	DGLGraph.adjacency_matrix(transpose, scipy_fmt="csr").

  dgl_warning('DGLGraph.adjacency_matrix_scipy is deprecated. '
/DATATWO/users/burouj/miniconda3/envs/nag-sim/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1272: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "



Graphs dropped for being too small: 9/800


In [54]:
max_degree = 0
for idx, graph in enumerate(graphs):
    if idx in idx_dropped:
        continue
    dense_adj = to_dense_adj(graph[0].coalesce().indices()).squeeze(0)
    local_max_degree = dense_adj.sum(dim=0).max()
    if local_max_degree > max_degree:
        max_degree = int(local_max_degree.item())

In [65]:
list_one_hot_features = list()
for idx, graph in enumerate(graphs):
    if idx in idx_dropped:
        list_one_hot_features.append(None)
        continue
    dense_adj = to_dense_adj(graph[0].coalesce().indices()).squeeze(0)
    degrees = dense_adj.sum(dim=0)
    num_nodes = graph[1].size(0)
    one_hot_features = torch.zeros(size=(num_nodes, max_degree+1)) # +1 for accomodating 0 degree
    for node in range(num_nodes):
        one_hot_features[node, int(degrees[node])] = 1
    list_one_hot_features.append(one_hot_features)

In [68]:
for idx, graph in enumerate(graphs):
    if idx in idx_dropped:
        continue
    graph[1] = torch.concat([graph[1], list_one_hot_features[idx]], dim=1)

In [71]:
graphs[0][1].size()

torch.Size([8, 10])